In [1]:
import numpy as np
from PIL import Image, ImageDraw
import torch
import torchvision.transforms as T
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import tqdm
from object_detection_data import ObjectDetectionDataset, Encoder, TransformedObjectDetectionDataset
import torch.nn as nn
from object_detection_models import MultiClassJetNet, LightningMultiClassJetNet
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
import os
torch.manual_seed(2)


2023-02-02 13:25:30.600477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 13:25:30.747557: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-02 13:25:31.696875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-02 13:25:31.696946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

# TODO
- Select the correct scalings for the default maps
- Select the correct initial learning rate
- Add learning rate schedule
- Log additional metrics
- Benchmark training loop
- Add validation data set
- Show Network predictions for example images
- Add data augmentation
- Make use of torch compile?
- Refactor the data preprocessing

In [2]:
default_scalings = [torch.tensor(
    [0.25, 0.25]), torch.tensor(
    [0.125, 0.125]), torch.tensor([0.125 / 2, 0.125 / 2]), torch.tensor([0.125 / 4, 0.125 / 4])]
feature_map_size = (8, 10)
num_classes = 4
encoder = Encoder(default_scalings, feature_map_size, num_classes)
data_path = 'SPLObjDetectDatasetV2/train'
transformed_train_data = TransformedObjectDetectionDataset(
    'SPLObjDetectDatasetV2/train', encoder)
transformed_val_data = TransformedObjectDetectionDataset(
    'SPLObjDetectDatasetV2/val', encoder)
train_loader = DataLoader(transformed_train_data, batch_size=16,
                          shuffle=True, num_workers=4)
val_loader = DataLoader(transformed_val_data, batch_size=16,
                        shuffle=False, num_workers=4)


In [3]:
pl_model = LightningMultiClassJetNet(num_classes, len(default_scalings))
trainer = pl.Trainer(limit_predict_batches=100,
                     max_epochs=10, callbacks=RichProgressBar())
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name  ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model │ MultiClassJetNet │ 27.1 K │
└───┴───────┴──────────────────┴────────┘

Trainable params: 27.1 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 27.1 K                                                                                               
Total estimated model params size (MB): 0

Output()

/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:48
8: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you 
turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(

`Trainer.fit` stopped: `max_epochs=10` reached.


In [7]:

transforms = T.Compose([T.PILToTensor(), T.ConvertImageDtype(
    torch.float32), T.Normalize([0.3333, 0.4772, 0.3155], [0.0905, 0.1207, 0.1082]), T.Grayscale(), T.Resize((60, 80))])

default_scalings = [torch.tensor(
    [0.25, 0.25]), torch.tensor(
    [0.125, 0.125]), torch.tensor([0.125 / 2, 0.125 / 2]), torch.tensor([0.125 / 4, 0.125 / 4])]
feature_map_size = (8, 10)
num_classes = 4
encoder = Encoder(default_scalings, feature_map_size, num_classes)
train_data_path = 'SPLObjDetectDatasetV2/train'
val_data_path = 'SPLObjDetectDatasetV2/val'
train_data = ObjectDetectionDataset(
    train_data_path, encoder, transforms=transforms)
validation_data = ObjectDetectionDataset(
    val_data_path, encoder, transforms=transforms)
train_loader = DataLoader(train_data, batch_size=16,
                          shuffle=True, num_workers=6)
val_loader = DataLoader(validation_data, batch_size=16,
                          shuffle=False, num_workers=6)
